# Spotify x Billboard Five Year Analysis Project: Exploratory Analysis

## Import packages 

In [1]:
import os
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from wordcloud import WordCloud
import fontTools
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

### Set Display Options

#### Set to display all columns (max 500)

In [2]:
pd.set_option('display.max_columns', 500)

#### Disable display window scrolling for notebook

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
   return false;
}

<IPython.core.display.Javascript object>

## Import dataframes

### Import All Chart Tracks Table

In [4]:
df1=pd.read_csv("../data/AllChartAlbumTracksRecoded.csv")

### Import Master Chart Table

In [5]:
df2=pd.read_csv("../data/MasterChartTableRecoded.csv")

In [6]:
position_int = list(df2['position'])
df2['chart_position'] = position_int

In [7]:
df3 = df2[["playlist_id", "playlist_name", "chart_position", "album_release_date", "release_year", "release_date_datetime", "album_id", 'album_artist']]

### Join Master Chart and All Track Tables on the Album ID

In [8]:
df4 = pd.merge(df3, df1, on='album_id')

In [9]:
df4 = df4.drop("Unnamed: 0", axis=1)

In [11]:
df4.to_csv('../data/AllTracksAndChartsJoined.csv')

#### Select five most popular songs for each album on chart

In [12]:
df5 = df4.sort_values('track_popularity', ascending = False).groupby(['playlist_name', 'album_id']).head(5)

In [13]:
df5.to_csv('../data/MostPopularAlbumTracksAllChartsJoined.csv')

## Explorations

### Preliminary Questions

#### Shape of the dataframe:

In [ ]:
df4.shape
#(rows, columns)

#### Most popular song in dataset:

In [ ]:
np.max(df4['track_popularity'])

In [ ]:
df4.loc[df4['track_popularity'] == 100]

In [ ]:
print(f"Most popular song: {set(df4[df4['track_popularity'] == 100]['track_name'].tolist())}" f" by {set(df4[df4['track_popularity'] == 100]['track_artist'].tolist())}\n")

#### Scatter plot of track popularity compared to valence

In [ ]:
sns.scatterplot(x="valence", y="track_popularity",
                hue="key",
                size="mode",
                palette="tab10",
                sizes=(150, 50), linewidth=0,
                data= df5[df5['playlist_name'] == 'Billboard 200 Top Albums 2021'])
plt.title('Scatter Plot of Valence vs Track Popularity Sized by Mode for Most Popular Songs from Chart Albums')
plt.xlabel('Valence')
plt.ylabel('Track Popularity')
vpm_plot_key=plt.legend()
vpm_plot_key.get_texts()[0].set_text('Key')
vpm_plot_key.get_texts()[1].set_text('C')
vpm_plot_key.get_texts()[2].set_text('C#')
vpm_plot_key.get_texts()[3].set_text('D')
vpm_plot_key.get_texts()[4].set_text('D#')
vpm_plot_key.get_texts()[5].set_text('E')
vpm_plot_key.get_texts()[6].set_text('F')
vpm_plot_key.get_texts()[7].set_text('F#')
vpm_plot_key.get_texts()[8].set_text('G')
vpm_plot_key.get_texts()[9].set_text('G#/Ab')
vpm_plot_key.get_texts()[10].set_text('A')
vpm_plot_key.get_texts()[11].set_text('A#/Bb')
vpm_plot_key.get_texts()[12].set_text('B')
vpm_plot_key.get_texts()[13].set_text('Mode')
vpm_plot_key.get_texts()[14].set_text('Minor Key')
vpm_plot_key.get_texts()[15].set_text('Major Key')
plt.savefig('../viz/pyplots/ValencePopularityModeScatterPlot.png')

#### Average loudness for all chart years together: 

In [ ]:
print(f"Average loudness for all years: {np.mean(df4['loudness'])} dB\n")

#### Average loudness for each chart year: 

In [ ]:
print(f"Average loudness in 2017: {np.mean(df4[df4['playlist_name'] == 'Billboard 200 Top Albums 2017']['loudness'])} dB")
print(f"Average loudness in 2018: {np.mean(df4[df4['playlist_name'] == 'Billboard 200 Top Albums 2018']['loudness'])} dB")
print(f"Average loudness in 2019: {np.mean(df4[df4['playlist_name'] == 'Billboard 200 Top Albums 2019']['loudness'])} dB")
print(f"Average loudness in 2020: {np.mean(df4[df4['playlist_name'] == 'Billboard 200 Top Albums 2020']['loudness'])} dB")
print(f"Average loudness in 2021: {np.mean(df4[df4['playlist_name'] == 'Billboard 200 Top Albums 2021']['loudness'])} dB")

In [ ]:
loudness_frame = df4[['track_id', 'playlist_name', 'main_genre', 'loudness', 'energy']]
average_loudness = list(loudness_frame.groupby('playlist_name')['loudness'].mean())
average_loudness

#### Chart of change in average loudness over time

In [ ]:
years = [2017, 2018, 2019, 2020, 2021]
plt.plot(years, average_loudness, color='green', marker='o', linestyle='solid')
plt.title("Change in Average Chart Album Track Loudness Over Time")
plt.xlabel("Year")
plt.ylabel("Average Loudness in dB normalization")
plt.xticks(np.arange(min(years), max(years)+1, 1.0))
plt.show()
plt.savefig('../viz/pyplots/LoudnessMeansOverTimeLineChart.png')

#### Error bar of average loudness by genre

In [ ]:
# create new table
all_genre_features_table = df4[['track_name', 'track_id', 'main_genre', 'loudness', 'valence']]
# grouping loudness by genre
genre_loudness = all_genre_features_table.groupby('main_genre')['loudness'].aggregate(['min', np.mean, 'max'])
# merge
genre_loudness_error = pd.merge(all_genre_features_table, genre_loudness, how='left', left_on='main_genre', right_on = 'main_genre', copy=False).fillna(0)

In [ ]:
# Plot
plt.errorbar(genre_loudness_error["main_genre"],genre_loudness_error["mean"], [genre_loudness_error["mean"] - genre_loudness_error['min'],genre_loudness_error['max']-genre_loudness_error["mean"]], linestyle='None',marker='o')
plt.xticks(rotation='vertical')
plt.title('Error Bar Chart of Loudness by Genre')
plt.xlabel('Genre')
plt.ylabel('Loudness in dB Normalization')
plt.show()
plt.savefig('../viz/pyplots/LoudnessbyGenreErrorChart.png')

#### Average valence for most popular album tracks all chart years together: 

In [ ]:
print(f"Average valence for all years: {np.mean(df5['valence'])} dB\n")

#### Error bar of average valence by genre

In [ ]:
# create new table
all_genre_features_table = df4[['track_name', 'track_id', 'main_genre', 'loudness', 'valence']]
# grouping loudness by genre
genre_valence = all_genre_features_table.groupby('main_genre')['valence'].aggregate(['min', np.mean, 'max'])
# merge
genre_valence_error = pd.merge(all_genre_features_table, genre_valence, how='left', left_on='main_genre', right_on = 'main_genre', copy=False).fillna(0)

In [ ]:
# Plot
plt.errorbar(genre_valence_error["main_genre"],genre_valence_error["mean"], [genre_valence_error["mean"] - genre_valence_error['min'],genre_valence_error['max'] - genre_valence_error["mean"]], linestyle='None',marker='o')
plt.xticks(rotation='vertical')
plt.title('Error Bar Chart of Valence by Genre')
plt.xlabel('Genre')
plt.ylabel('Valence')
plt.show()
plt.savefig('../viz/pyplots/ValencebyGenreErrorChart.png')

#### Count plot for songs in each major and minor key

In [ ]:
key_plot = sns.countplot(data=df4, x='key', hue='mode', palette = 'Set1')
plt.xlabel('Key')
plt.ylabel('Song Count')
plt.title('Count of Songs in Major and Minor Keys Across All Album Tracks')
key_plot.set_xticklabels(['C','C#/Db','D','D#/Eb','E','F','F#/Gb','G', 'G#/Ab', 'A', 'A#/Bb', 'B'])
plot_modes=plt.legend()
plot_modes.get_texts()[0].set_text('Minor Key')
plot_modes.get_texts()[1].set_text('Major Key')
plt.savefig('../viz/pyplots/KeyCountPlot.png')

#### Number of tracks with explicit lyrics

In [ ]:
df4['track_explicit'].value_counts()

In [ ]:
plt.pie(df4['track_explicit'].value_counts(), 
        explode=(0.1, 0), 
        colors=['green', 'yellow'], 
        autopct='%1.1f%%',
        shadow=True,
        startangle=180, labels = ['Does Not Have Parental Advisory Label or Status Unknown', 'Has Parental Advisory Label']);
plt.title('Percentage of Chart Album Tracks with Parental Advisory Label')
plt.savefig('../viz/pyplots/ExplicitPieChart.png')

#### Explicit Lyrics by Genre

In [ ]:
# Slice dataset
exp_track = df4[['main_genre', 'track_explicit']]
#Dummy code the explicit column
exp_dummy = pd.get_dummies(exp_track, prefix='exp', columns=['track_explicit'])
# Create grouped df
exp_dummy_grouped = exp_dummy.groupby(
     ['main_genre']
 ).agg(
     not_exp = ('exp_False','sum'),
     exp = ('exp_True','sum'),
 ).reset_index()

In [ ]:
# Create stacked bar chart
labels = list(exp_dummy_grouped['main_genre'])
ax = exp_dummy_grouped.plot(kind='bar', stacked=True)
ax.set(xticklabels = (labels))
plt.title("Explicit Tracks by Genre")
plt.xlabel("Genre")
plt.ylabel("Number of Tracks")
plt.savefig('../viz/pyplots/ExpGenreStackedBarChart.png')

#### Stacked to 100% (filled) bar chart 

In [ ]:
plotdata = exp_dummy_grouped.set_index('main_genre')

In [ ]:
stacked_data = plotdata.apply(lambda x: x*100/sum(x), axis=1)
labels = list(exp_dummy_grouped['main_genre'])
ax = stacked_data.plot(kind='bar', stacked=True)
ax.set(xticklabels = (labels))
plt.title("Explicit Tracks by Genre")
plt.xlabel("Genre")
plt.ylabel("Percentage of Tracks")
plt.savefig('../viz/pyplots/ExpGenreFilledBarChart.png')

#### Explicit Lyrics by Year

In [ ]:
# Slice dataset
exp_year = df4[['playlist_name', 'track_explicit']]
#Dummy code the explicit column
exp_year_dummy = pd.get_dummies(exp_year, prefix='exp', columns=['track_explicit'])
# Create grouped df
exp_year_dummy_grouped = exp_year_dummy.groupby(
     ['playlist_name']
 ).agg(
     not_exp = ('exp_False','sum'),
     exp = ('exp_True','sum'),
 ).reset_index()
exp_year_dummy_grouped

In [ ]:
# Create stacked bar chart
labels = list(['2017', '2018', '2019', '2020', '2021'])
ax = exp_year_dummy_grouped.plot(kind='bar', stacked=True, color=['darkblue', 'red'])
ax.set(xticklabels = (labels))
plt.title("Explicit Tracks by Year")
plt.xlabel("Year")
plt.ylabel("Number of Tracks")
plt.savefig('../viz/pyplots/ExpYearStackedBarChart.png')

#### Stacked to 100% (filled) bar chart 

In [ ]:
plotdata = exp_year_dummy_grouped.set_index('playlist_name')
plotdata.head()

In [ ]:
stacked_data = plotdata.apply(lambda x: x*100/sum(x), axis=1)
labels = list(['2017', '2018', '2019', '2020', '2021'])
ax = stacked_data.plot(kind='bar', stacked=True, color=['darkblue', 'red'])
ax.set(xticklabels = (labels))
plt.title("Explicit Tracks by Chart Year")
plt.xlabel("Year")
plt.ylabel("Percentage of Tracks")
plt.savefig('../viz/pyplots/ExpYearFilledBarChart.png')

#### Average valence for most popular album songs for each chart year: 

In [ ]:
print(f"Average valence in 2017: {np.mean(df5[df5['playlist_name'] == 'Billboard 200 Top Albums 2017']['valence'])}")
print(f"Average valence in 2018: {np.mean(df5[df5['playlist_name'] == 'Billboard 200 Top Albums 2018']['valence'])}")
print(f"Average valence in 2019: {np.mean(df5[df5['playlist_name'] == 'Billboard 200 Top Albums 2019']['valence'])}")
print(f"Average valence in 2020: {np.mean(df5[df5['playlist_name'] == 'Billboard 200 Top Albums 2020']['valence'])}")
print(f"Average valence in 2021: {np.mean(df5[df5['playlist_name'] == 'Billboard 200 Top Albums 2021']['valence'])}")

#### Mean values of each audio feature

In [ ]:
df4[['danceability', 'energy', 'acousticness', 'instrumentalness', 'liveness', 'valence']].mean()

#### Bar plot of audio feature means of most popular tracks per album

In [ ]:
feature_bars = df4[['danceability', 'energy', 'acousticness', 'instrumentalness', 'liveness', 'valence']]
feature_bars.mean().plot.bar()
plt.title('Mean Values of Audio Features Across All Tracks')
plt.xlabel('Audio Features')
plt.ylabel('Values')
plt.savefig('../viz/pyplots/MeanAudioFeaturesBarPlot.png')

#### Histograms of audio features

In [ ]:
features = ['danceability', 'energy', 'key', 'loudness', 'acousticness', 'liveness', 'valence','tempo']
for col in features:
  sns.displot(df5, x=col, kde=True)
  plt.show()

#### Correlation Heatmap of Features

In [ ]:
# Dataframe with audio features
track_feature_corr = df4[['playlist_name', 'chart_position', 'track_popularity', 'track_explicit', 'danceability', 'energy', 'key','loudness', 'mode', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]

# Generate a mask for the upper part of the chart
mask = np.triu(np.ones_like(track_feature_corr.corr(), dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 8))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Can add mask=mask in sns.heatmap to mask top triangle of plot. Removed to view entire plot.
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(track_feature_corr.corr(), cmap=cmap, vmin=0, mask=mask, vmax=.5, 
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.savefig('../viz/pyplots/CorrHeatMapAllTracks.png')

#### Average song length

In [ ]:
print(f"Average song duration in minutes for all years: {int(np.mean(df4['duration_ms']) // 60000)}:" f"{round(np.mean(df4['duration_ms'] / 10000) % 60)} \n")

In [ ]:
print(f"Average song duration in minutes for all years: {int(np.mean(df4['duration_ms']) // 60000)}:" f"{round(np.mean(df4['duration_ms'] / 10000) % 60)} \n")

#### Longest song

In [ ]:
print(f"Longest song in minutes: {df4.iloc[df4['duration_ms'].idxmax()]['track_name']} by "
        f"{df4.iloc[df4['duration_ms'].idxmax()]['track_artist']}, length = "
        f"{int(df4['duration_ms'].max() // 60000)}:" f"{round((df4['duration_ms'].max() / 10000) % 60)}\n")

#### Artist that has the most appearances on charts across the entire dataset year range

In [ ]:
df_freq_art = df2.dropna(subset=['album_artist'])
print(f"Most frequently appearing album artist: {df_freq_art['album_artist'].value_counts().idxmax()}" f" appears {df_freq_art['album_artist'].value_counts().max()} times.\n")

#### Number of chart appearances by particular artists:

In [ ]:
print("Number of chart appearances of albums by Taylor Swift: " + str(df2['album_artist'].str.contains("Taylor Swift").sum()))
print("Number of chart appearances of albums by Queen: " + str(df2['album_artist'].str.contains("Queen").sum()))
print("Number of chart appearances of albums by Elton John: " + str(df2['album_artist'].str.contains("Elton John").sum()))

#### Create word cloud viz of artist chart appearance frequency

Create df of artist frequencies

In [ ]:
album_artist_chart_count = df2['album_artist'].value_counts()
album_artist_chart_count = album_artist_chart_count.reset_index()
album_artist_chart_count.columns = ['artist_name', 'artist_chart_frequency'] # change column names

Convert df to dict

In [ ]:
artist_freq = dict(zip(album_artist_chart_count['artist_name'].tolist(), album_artist_chart_count['artist_chart_frequency'].tolist()))

Plot word cloud

In [ ]:
wc_white = WordCloud(background_color='white',width=800, height=400, max_words=100).generate_from_frequencies(artist_freq)
plt.figure(figsize=(20, 10))
plt.imshow(wc_white, interpolation='bilinear')
plt.axis('off')
plt.show()
# Plot visualizations must be saved manually as png, but can be exported to svg with the following:
# wordcloud_svg = wc_white.to_svg(embed_font=True)
# f = open("../viz/pyplots/ArtistChartAppearanceWordCloudWhite.svg","w+")
# f.write(wordcloud_svg )
# f.close()

In [ ]:
wc_black = WordCloud(background_color='black',width=800, height=400, max_words=100).generate_from_frequencies(artist_freq)
plt.figure(figsize=(20, 10))
plt.imshow(wc_black, interpolation='bilinear')
plt.axis('off')
plt.show()
# Plot visualizations must be saved manually as png, but can be exported to svg with the following:
# wordcloud_svg = wc_black.to_svg(embed_font=True)
# f = open("../viz/pyplots/ArtistChartAppearanceWordCloudBlack.svg","w+")
# f.write(wordcloud_svg )
# f.close()

#### Queen songs appearances on all charts

In [ ]:
df2.loc[df2['album_artist'] == "Queen"]

#### Most popular Taylor Swift song on all charts

In [ ]:
print("Most popular Taylor Swift song: " + str(df4.iloc[df4[df4['track_artist'].str.contains('Taylor Swift')]['track_popularity'].idxmax]['track_name']) + "\n")

#### Oldest track

Wrangle release date format

In [ ]:
# album_release_frame = df4[['track_name', 'track_artist', 'album_release_date', 'release_date_datetime', 'release_year']]
album_release_frame = df4[['album_release_date', 'track_name', 'track_artist']]
album_release_frame['datum'] = pd.to_datetime(album_release_frame['album_release_date'], format='%Y-%m-%d')
album_release_frame['year'] = album_release_frame['datum'].dt.year

In [ ]:
print(f" The oldest track present on the charts is \"{album_release_frame.loc[album_release_frame['year'].idxmin()]['track_name']}\" released in {df2['album_release_date'].min()}.\n")

In [ ]:
# Recode newest date to string
date_time_str = album_release_frame.loc[album_release_frame['datum'].idxmax()]['datum'].strftime("%B %-d, %Y")
print(f" The newest track present on the charts is \"{album_release_frame.loc[album_release_frame['datum'].idxmax()]['track_name']}\" released by {album_release_frame.loc[album_release_frame['datum'].idxmax()]['track_artist']} on {date_time_str}.\n")


#### Bar chart of songs by release year

In [ ]:
# grouping the songs by year to get the sum
release_year = album_release_frame.groupby('year', as_index=False)['track_name'].count()

In [ ]:
sns.barplot(data=release_year, x='year', y='track_name', palette='mako_r')
plt.xticks(rotation=90)
plt.title('Number of Songs by Album Release Year')
plt.xlabel('Release Year')
plt.ylabel('Number of Tracks')
plt.show()
plt.savefig('../viz/pyplots/TracksPerReleaseYearBarChart.png')